# Advanced Legal Analytics
# LAW 3027 Tutorial 5: Large Language Models for Legal Text Summarization


#### Intended Learning Outcomes:
This notebook provides an introduction to the use of large language models for summarizing (legal) texts.
By the end of this notebook you will know how to:
- Apply large language models (LLMs) to summarize legal texts.
- Utilize different kinds of prompting to generate summaries of legal texts.
- Evaluate the generated summaries based on state-of-the-art summarization evaluation metrics such as ROUGE.


#### Reading Material:
-  paper on Roleplay Prompting here: https://arxiv.org/abs/2308.07702
-  ROUGE metric: https://youtu.be/TMshhnrEXlg 
- [OpenAI API](https://platform.openai.com/docs/api-reference/introduction)
- [OpenAI prompt engineering guidelines](https://platform.openai.com/docs/guides/prompt-engineering)


#### Installation

Run this command to install the required packages for this lesson:

```bash
pip install datasets pandas openai rouge
```

If you are on colab, you can do this by running the cell below.

In [ ]:
!pip install datasets pandas openai rouge

In [ ]:
import pandas as pd
import datasets
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

#### Setup
We will be prompting a Mistral AI model, and in particular the **Mixtral 8x7b**. We will use the OpenAI API connected to our own custom server. 

In [ ]:
# Enter your key and initialize the client in this cell
import openai

MODEL_MIXTRAL = "Mixtral-8x7B-Instruct-v0.1"
MODEL_EMBEDDING = "text-embedding-3-small"

key = "" # your key here
base_url = "https://llm.wstrmnn.com"

client = openai.OpenAI(
    api_key=key,
    base_url=base_url,
)

#### 1.1 Let's start with Summarization of some Legal Texts

Let's read the Article 10 of the AI Act which focusses on Data and Data Governance. You can read it in a better format here:https://artificialintelligenceact.eu/article/10/ 

Below, we created a string variable, `Art10_AIAct` by copying and pasting the text of Article 10 into this variable.

In [ ]:
Art10_AIAct = """
1. High-risk AI systems which make use of techniques involving the training of models with data shall be developed on the basis of training, validation and testing data sets that meet the quality criteria referred to in paragraphs 2 to 5.

2.Training, validation and testing data sets shall be subject to appropriate data governance and management practices. Those practices shall concern in particular,

(a)the relevant design choices;

(b)data collection;

(c)relevant data preparation processing operations, such as annotation, labelling, cleaning, enrichment and aggregation;

(d)the formulation of relevant assumptions, notably with respect to the information that the data are supposed to measure and represent;

(e)a prior assessment of the availability, quantity and suitability of the data sets that are needed;

(f)examination in view of possible biases;

(g)the identification of any possible data gaps or shortcomings, and how those gaps and shortcomings can be addressed.

3.Training, validation and testing data sets shall be relevant, representative, free of errors and complete. They shall have the appropriate statistical properties, including, where applicable, as regards the persons or groups of persons on which the high-risk AI system is intended to be used. These characteristics of the data sets may be met at the level of individual data sets or a combination thereof.

4.Training, validation and testing data sets shall take into account, to the extent required by the intended purpose, the characteristics or elements that are particular to the specific geographical, behavioural or functional setting within which the high-risk AI system is intended to be used.

5.To the extent that it is strictly necessary for the purposes of ensuring bias monitoring, detection and correction in relation to the high-risk AI systems, the providers of such systems may process special categories of personal data referred to in Article 9(1) of Regulation (EU) 2016/679, Article 10 of Directive (EU) 2016/680 and Article 10(1) of Regulation (EU) 2018/1725, subject to appropriate safeguards for the fundamental rights and freedoms of natural persons, including technical limitations on the re-use and use of state-of-the-art security and privacy-preserving measures, such as pseudonymisation, or encryption where anonymisation may significantly affect the purpose pursued.

6.Appropriate data governance and management practices shall apply for the development of high-risk AI systems other than those which make use of techniques involving the training of models in order to ensure that those high-risk AI systems comply with paragraph 2. 
"""

#### 1.2 Write a function to generate a summary for any given text. Use this function to generate a summary for the Article 10 of the AI Act.

The function should be called `generate_summary()`. It should have two input arguments:

-`system_instructions`: Instructions to the model


-`user_text`: A message from the user. Here you can provide the text for which the summary needs to be generated.


**The function should `return` the generated summary by the LLM. Please set the `max_tokens` = 200 because your key has a limitation of the number of tokens. Therefore, we do not recommend that you generate long summaries with this model.**


In [ ]:
def generate_summary(system_instructions, user_text):
    #your code here

    return response.choices[0].message.content

#### 1.2.1 Now apply the function `generate_summary()` to generate a summay for the Article 10 of the AI Act

Read the summary and check if its meaningful. You can try a different prompt and see if the summary changes or not.

In [ ]:
system_instructions = "" #your system prompt here

generate_summary(system_instructions,Art10_AIAct) #call the function

### 1.3 Roleplay Prompting
One of the simplest prompt engineering techniques is to specify a role for the LLM. Accoding to Kong et al. (2023),  modern LLMs, with their sophisticated role-playing abilities, have greatly enhanced user experiences and created innovative modes of interaction. They can adeptly emulate diverse personas, from fictional characters to historical and contemporary figures. The assigned role offers context regarding the LLM’s identity and background. By adopting this persona, the LLM can deliver more authentic, in-character responses tailored to that role.

Kong, A., Zhao, S., Chen, H., Li, Q., Qin, Y., Sun, R., & Zhou, X. (2023). Better zero-shot reasoning with role-play prompting. arXiv preprint arXiv:2308.07702. 

Read the paper on Roleplay Prompting here: https://arxiv.org/abs/2308.07702 

**1.3.1 Now let's assume that for summarizing Article 10 of the AI Act, you want to assign LLM the role of a renowned expert in AI regulations. Draft a prompt for the same and generate the summary. How does this summary compare with the summaries generated with the previous prompt?**

In [ ]:
#Now with role play prompting
system_instructions = "" #your roleplay prompt here

generate_summary(system_instructions,Art10_AIAct)

#### 1.3.2 Which summary was the best according to you and why?

#### 1.3.3 Generating a summary for a particular contextual situation
Let's try to generate summaries for some very specific questions a person may have about the Article 10 of the AI Act. An example is provided below. But you can also try some other specific questions or contexts. Check the summary generated by the LLM using the `generate_summary()` function. 

```
system_prompt_bias = '''
You are an AI and law expert when it comes to AI Act. A person asks you the following question:
'I want to create an AI system but I am afraid there may be biases in the dataset.'

Create a summary of article 10 (provided below), as it applies to their situation. Only include elements directly related to the question.
'''
```

In [ ]:
sys_prompt_bias = '''
You are an AI and law expert when it comes to AI Act. A person asks you the following question:
'I want to create an AI system but I am afraid there may be biases in the dataset.'

Create a summary of article 10 (provided below), as it applies to their situation. Only include elements directly related to the question.
'''

In [ ]:
generate_summary(sys_prompt_bias,Art10_AIAct)

#### 1.4 Working with a Legal Dataset for Text Summarization
#### BillSum: A Corpus for Automatic Summarization of US Legislation

The paper is available here: https://aclanthology.org/D19-5406/

In this paper, the authors introduce the first dataset for summarization of US Congressional and California state bills. They also benchmark some state-of-the-art language models for text summarization of bills.

#### 1.4.1 Load the dataset

In [ ]:
dataset_billsum = load_dataset("billsum")

#### 1.4.2 From the train Dataset, extract the `text` and `summary` columns and store them in a pandas DataFrame called `df_billsum`

In [ ]:
# create a DataFrame with summary and text features
df_billsum = pd.DataFrame({
    'text': dataset_billsum['train']['text'],
    'summary': dataset_billsum['train']['summary']
})
df_billsum.head()

#### 1.4.3 Count the number of words in the `text` column. Also compute the length of the `text` column
Since, we have limitations with the number of tokens for each API key and summarization consumes a large number of tokens, we will restrict our summarization exercises only to the Bills in the dataset which have the shortest string length or the lowest number of words.

Therefore, you need to write code to add two more columns to the dataframe `df_billsum`. These two columns are called `word_count` and `string_length`. The `word_count` should contain the number of words in the `text` column. This can be obtained by using the `split()` function which splits a string into a list of words, using any whitespace characters. The `string_length` is the length of the `text` column string.

#### 1.4.5 Use the `describe()` function of `pandas` to get some statistics about the `word_count` and `string_length`.

#### 1.4.6 Based on your understanding of these statistics, create a new dataframe `df_billsum_small` such that it contains only 4 rows. You can do this in one of the two ways. Follow only one of the options below:
- pick the 4 rows where the `string_length` of the `text` column is minimum
  
   <strong>OR</strong>
   
- pick the 4 rows where the `word_count` is the lowest 4 values in the `word_count` column,

#### 1.5 Data Cleaning

Lets try to read one random summary from `df_billsum_small`and see how does it look when we print it.
Do you see whitespaces and newline characters?  The newline characters are represented by `\n`

#### 1.6 Write a function to replace the newlines and multiple whitespaces by whitespace in the `text` and `summary` columns of the dataframe `df_billsum_small`

Hint: Here again you may use the `split()` function or also other solutions are ok.

#### 1.7 Print a clean summary from any of the 4 rows for better reading. 

#### 1.8 Apply Prompt on the dataframe

Write a function `apply_prompt()` to apply a prompt on the dataframe. The function should take the following 3 arguments:

- `system_instructions`: Instructions to the model

- `dataframe`: any pandas data frame that should contain a column `text`

- `new_column_name` : the new column name where the output of LLM will be stored

So we want to prompt the LLM using `system_instructions` and some `text` from the `dataframe` and then store the output in a `new_column_name`


In [ ]:
def apply_prompt(system_instructions, dataframe, new_column_name):
    #your code here

#### 1.8.1 Use the apply_prompt() function to generate summary from the LLM on the `text` column of `df_billsum_small`. The new column in which the LLM Generated Summary should be stored should be called `LLM_Generated_Summary`. 

In [ ]:
summary_prompt = "" #your prompt here
apply_prompt(summary_prompt, df_billsum_small, 'LLM_Generated_Summary')

#### 1.9 Check the dataframe after calling the `apply_prompt()` function. See if you have the new column `LLM_Generated_Summary`.

Print one of the `text` and the corresponding `LLM_Generated_Summary`. Read the summary and assess if its appropriate. You can do this manually of course.  

#### 2.0 Evaluation of the Generated Text Summary

We will focus on ROUGE which is a state-of-the-art metric used for evaluation of the text summarization task. 

#### ROUGE

Rouge scores are used to evaluate the quality of generated summaries by comparing them to reference (human) summaries. 

**Video to learn about ROUGE Score: Please watch this video to understand about the ROUGE metric: https://youtu.be/TMshhnrEXlg . Watch the video only till time 3:20. We will compute the rouge score using another library than the video because it works directly with dataframes**. 

 
 - ROUGE-1: Measures the overlap of unigrams (single words) between the generated and reference summaries.
 - ROUGE-2: Measures the overlap of bigrams (pairs of words) between the generated and reference summaries.
 - ROUGE-L: Focuses on the longest common subsequence (LCS), assessing the longest string of words that appears in both summaries in the same order.


#### 2.1 Based on the example in the video, compute the ROUGE score for the following two texts

```
from rouge import Rouge
rouge = Rouge()

machine_generated_summary = "I really loved reading the Hunger Games"

human_summary = "I loved reading the Hunger Games"

scores = rouge.get_scores(generated_summary, human_summary)
scores
```
Verify the rouge scores you have computed with the ones in the video.


In [ ]:
#example computation of rouge as per huggingface
from rouge import Rouge
rouge = Rouge()


machine_generated_summary = "I really loved reading the Hunger Games"
human_summary = "I loved reading the Hunger Games"
scores = rouge.get_scores(machine_generated_summary, human_summary)
scores

#### 2.2 Let's create a function called `compute_rouge_scores()` for this. The function takes two input arguments. Each argument can be a column of the dataframe. 

- The first one is the `generated_summary`.
- The second one is `human_summary`

Let's say the function returns only the F1-score of ROUGE

In [ ]:
def compute_rouge_scores(generated_summary, human_summary):
    rouge = Rouge()
    scores = rouge.get_scores(generated_summary, human_summary, avg=True) # a single dict with average values
    return [scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f']] #just taking f1-scores for convenience

#### 2.3 Now compute the ROUGE metrics between the columns `LLM_Generated_Summary` and `summary` of `df_billsum_small` using the `compute_rouge_scores()` function

In [ ]:
human_summary_billsum= df_billsum_small['summary']
generated_summary_billsum = df_billsum_small['LLM_Generated_Summary'] 
result = compute_rouge_scores(generated_summary_billsum,human_summary_billsum)
result

#### 2.4 Plot the ROUGE scores in a bar plot

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.bar(['Rouge-1','Rouge-2','Rouge-l'], result, color=['red', 'green', 'blue'])
plt.title('Evaluation of Summary')
plt.show()

#### Bonus Exercise (outside the tutorial for those who are interested)

What do you think are the limitations of the Rouge score when it comes to evaluating the summary generated by LLMs?